In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## 1. Load data

In [3]:
import pandas as pd

df = pd.read_csv("./data/connectpoint/data.csv")
df

,Objeto,Categoria
0,* Registro de preços * Contratação de empresa...,30101
1,CONTRATAÇÃO DE EMPRESA ESPECIALIZADA PARA FORN...,10110
2,* Registro de preços * * Licitação Eletrônica ...,10110
3,Contratação de empresa especializada para pres...,10110
4,- Contratação de empresa especializada para pr...,10110
...,...,...
4769,Pregão Eletrônico - Contratação de serviços co...,0
4770,"CONSTRUÇÃO DE QUADRA COBERTA E VESTIÁRIO, MODE...",10205
4771,REFERENTE A PAVIMENTAÇÃO EM CBUQ MAIS BASE DE ...,10205
4772,CONTRATAÇÃO DE EMPRESA PARA EXECUÇÃO DOS SERVI...,10205


## 2. Generate embeddings

In [5]:
import os
import google.generativeai as genai

genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

In [15]:
embeddings = genai.embed_content(model="models/text-embedding-004", content=df["Objeto"].values)

df["Embedding"] = embeddings["embedding"]
df

,Objeto,Categoria,Embedding
0,* Registro de preços * Contratação de empresa...,30101,"[0.027471611, 0.039454266, -0.038936675, -0.02..."
1,CONTRATAÇÃO DE EMPRESA ESPECIALIZADA PARA FORN...,10110,"[0.008966482, 0.04420506, -0.030004129, -0.013..."
2,* Registro de preços * * Licitação Eletrônica ...,10110,"[-0.03665017, 0.0008280597, -0.05525631, -0.01..."
3,Contratação de empresa especializada para pres...,10110,"[0.0012117622, 0.012949114, -0.023142496, -0.0..."
4,- Contratação de empresa especializada para pr...,10110,"[-0.017860573, 0.019950267, -0.054686528, -0.0..."
...,...,...,...
4769,Pregão Eletrônico - Contratação de serviços co...,0,"[-0.027466945, 0.031838894, -0.062915854, -0.0..."
4770,"CONSTRUÇÃO DE QUADRA COBERTA E VESTIÁRIO, MODE...",10205,"[-0.045839936, -0.0062474306, 0.0013476581, -0..."
4771,REFERENTE A PAVIMENTAÇÃO EM CBUQ MAIS BASE DE ...,10205,"[-0.013294374, -0.008769278, -0.03451485, -0.0..."
4772,CONTRATAÇÃO DE EMPRESA PARA EXECUÇÃO DOS SERVI...,10205,"[-0.019865897, 0.059927408, -0.016741052, -0.0..."


In [16]:
df.to_pickle("./data/connectpoint/data_embedded.pkl")

## 3. Train model

In [ ]:
import pandas as pd

df = pd.read_pickle("./data/connectpoint/data_embedded.pkl")
df

In [21]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

In [44]:
X = np.array(df["Embedding"].values.tolist())
y = df["Categoria"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [45]:
clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000)
clf.fit(X_train, y_train)

MLPClassifier(max_iter=1000)

In [48]:
y_pred = clf.predict(X_test)
clr = classification_report(y_test, y_pred)
print(clr)

              precision    recall  f1-score   support

           0       0.67      0.67      0.67         3
       10101       0.79      0.84      0.82       103
       10110       0.71      0.69      0.70        36
       10201       0.62      0.78      0.69        27
       10202       0.50      1.00      0.67         1
       10203       0.00      0.00      0.00         2
       10204       0.62      0.71      0.67         7
       10205       0.88      0.84      0.86       184
       10206       0.75      1.00      0.86         3
       10207       0.75      0.75      0.75         4
       10208       1.00      0.73      0.84        11
       10209       0.71      1.00      0.83        17
       10211       1.00      1.00      1.00         1
       10304       0.00      0.00      0.00         1
       10306       1.00      1.00      1.00         5
       10307       0.50      0.67      0.57         3
       10401       1.00      0.80      0.89         5
       20102       1.00    

/home/hellmrf/test/gemini/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hellmrf/test/gemini/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hellmrf/test/gemini/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

In [49]:
acc = (y_test == y_pred).sum() / len(y_test)
acc


np.float64(0.799163179916318)